In [1]:
import os 
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr



In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()


OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = """
You are a helpful assistant for a Resturant called The Diner.
You have to ask for the customer's name and then ask for their order.
You have to return the total price of the order. If they ask for a item that is not in the menu, you have to say that it is not in the menu.
Always be accurate and friendly.
"""




### Create a table of the menu items and their prices.

In [7]:

import sqlite3

DB = "menu.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS menu (item TEXT PRIMARY KEY, price REAL)')

In [8]:
def set_menu_item(item, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO menu (item, price) VALUES (?, ?) ON CONFLICT(item) DO UPDATE SET price = ?', (item.lower(), price, price))
        conn.commit()

In [ ]:
item_prices = {"burger": 5, "fries": 3, "soda": 2, "salad": 4, "ice cream": 3}
for item, price in item_prices.items():
    set_menu_item(item, price)
    

 Database Tool Calling

In [10]:
def get_menu_item(item):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM menu WHERE item = ?', (item.lower(),))
        result = cursor.fetchone()
        return result[0] if result else None


In [11]:
get_menu_item("burger")

5.0

In [ ]:
price_function = {
    "name": "get_menu_item",
    "description": "Get the price of a menu item.",
    "parameters": {
        "type": "object",
        "properties": {
            "item": {
                "type": "string",
                "description": "The name of the menu item to get the price of."
            }
        },
        "required": ["item"]
    }
}

